In [2]:
import csv
from socket import MSG_MCAST
from urllib.parse import quote, quote_plus
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [6]:
strDate = '20201101'
endDate = '20201130'

filename = f"daum_news_{strDate}-{endDate}.csv"
f = open(filename, "w", encoding="utf-8-sig", newline="")
writer = csv.writer(f)

headers = {"User-Agent" : "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.159 Safari/537.36"}

dt_index = pd.date_range(start=strDate, end=endDate)
dt_list = dt_index.strftime("%Y%m%d").tolist()
max_page = 10000 # 뉴스 페이지 탭 수 지정

for i in dt_list:
    print('날짜',i)
    for j in range(1, max_page):
        main_url = f"https://news.daum.net/breakingnews/?page={j}&regDate={i}" # url 입력
        res = requests.get(main_url, headers=headers)
 
        val_url = f"https://news.daum.net/breakingnews/?page={j+1}&regDate={i}"
        val = requests.get(val_url, headers=headers)


        if (res.status_code== 200) & (val.status_code == 200):
            print('page : ',j, main_url, 'status:',res.status_code)
            soup = BeautifulSoup(res.text, "lxml") # soup으로 저장
            val_soup = BeautifulSoup(val.text, "lxml")

            main = soup.find("ul", attrs={"class":"list_news2 list_allnews"})
            val_main = val_soup.find("ul", attrs={"class":"list_news2 list_allnews"})
            if main != val_main:
                news = main.find_all("strong", attrs={"class":"tit_thumb"})
                cnt = 0
                for new in news:
                    urls = new.select_one("a")["href"]# 페이지에 나와있는 뉴스 URL 변수 입력
                    # print(urls)
                    result = requests.get(urls, headers=headers)         # request 로 다시 개별 뉴스 접속
                    if result.status_code == 200:
                        news_soup = BeautifulSoup(result.text, "lxml")
                        # 뉴스 제목, 발행시간, 기사본문 저장
                        title = news_soup.find("h3", attrs={"tit_view"}).get_text().strip()
                        pubdate = news_soup.find("span", attrs={"num_date"}).get_text().strip()
                        text = news_soup.find("div", attrs={"news_view"}).get_text().strip()
                        cnt += 1
                        # print(j,'of',cnt,'번째 기사')
                        # print(i,j,'of',cnt,'번째 기사', urls,'status:', result.status_code)
                        writer.writerow([cnt, title, pubdate, urls, text])
                    else:
                        print(i,j,'of',cnt,'번째 기사','error_code :',result.status_code, urls)
                        pass
            else:
                news = main.find_all("strong", attrs={"class":"tit_thumb"})
                for new in news:
                    urls = new.select_one("a")["href"]# 페이지에 나와있는 뉴스 URL 변수 입력
                    # print(urls)

                    result = requests.get(urls, headers=headers)         # request 로 다시 개별 뉴스 접속
                    if result.status_code == 200:
                        news_soup = BeautifulSoup(result.text, "lxml")
                        # 뉴스 제목, 발행시간, 기사본문 저장
                        title = news_soup.find("h3", attrs={"tit_view"}).get_text().strip()
                        pubdate = news_soup.find("span", attrs={"num_date"}).get_text().strip()
                        text = news_soup.find("div", attrs={"news_view"}).get_text().strip()
                        cnt += 1
                        # print(j,'of',cnt,'번째 기사')
                        # print(i,j,'of',cnt,'번째 기사', urls,'status:', result.status_code)
                        writer.writerow([cnt, title, pubdate, urls, text])
                        break
                    else:
                        print(i,'page : ',j,'of',cnt,'번째 기사','error_code :',result.status_code, urls)
                        pass
        else:
            print(i,'page : ',j,'error_code :',res.status_code, main_url)
            pass

날짜 20201101
page :  1 https://news.daum.net/breakingnews/?page=1&regDate=20201101 status: 200
page :  2 https://news.daum.net/breakingnews/?page=2&regDate=20201101 status: 200
page :  3 https://news.daum.net/breakingnews/?page=3&regDate=20201101 status: 200
page :  4 https://news.daum.net/breakingnews/?page=4&regDate=20201101 status: 200
page :  5 https://news.daum.net/breakingnews/?page=5&regDate=20201101 status: 200
page :  6 https://news.daum.net/breakingnews/?page=6&regDate=20201101 status: 200
page :  7 https://news.daum.net/breakingnews/?page=7&regDate=20201101 status: 200
page :  8 https://news.daum.net/breakingnews/?page=8&regDate=20201101 status: 200
page :  9 https://news.daum.net/breakingnews/?page=9&regDate=20201101 status: 200
page :  10 https://news.daum.net/breakingnews/?page=10&regDate=20201101 status: 200
page :  11 https://news.daum.net/breakingnews/?page=11&regDate=20201101 status: 200
page :  12 https://news.daum.net/breakingnews/?page=12&regDate=20201101 status: 20

KeyboardInterrupt: 